<a href="https://colab.research.google.com/github/dkanh6/Computational_Neuroscience/blob/main/E1_Single_Compartment_Cell.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# E1 Single Compartment Cell in NEURON
This is an example created purely to familarize myself with the NEURON package and experiment with its many features. First I will install NEURON and some other useful libraries such as numpy and matlplotlib

In [4]:
!pip install neuron

In [5]:
from neuron import h
import matplotlib.pyplot as plt
import numpy as np
h.load_file('stdrun.hoc') # compiles some of the files for use in the simulation